This tool converts a folder of samples to a big rectangular matrix with one mono sample per row.

Samples should be placed in `data/mydataset/samples/`. They could be `.mp3`, `.wav`, or anything else that ffmpeg can work with. They may be all in one folder, or in nested sub-folders.

Change the path below to point to the root directory, e.g., `data/mydataset/`.

The samplerate `sr` is not necessarily the native samplerate of the samples, it's the samplerate you want to load them at.

The output of this notebook is:
* `data/mydataset/durations.txt`
* `data/mydataset/filenames.txt`
* `data/mydataset/samples.npy`

In [4]:
data_root = '/Users/zebra/Developer/Python/AudioNotebooks/data/drums'
sr = 44000
max_length = sr*4 # ignore samples longer than 4 seconds
fixed_length = sr/4 # trim all samples to 250 milliseconds
limit = None # set this to 100 to only load the first 100 samples

In [8]:
import numpy as np
from os.path import join
from utils import *
from multiprocessing import Pool
from list_all_files import list_all_files_with_subdirs

['/Users/zebra/Developer/Python/AudioNotebooks/data/drums/samples/.DS_Store', '/Users/zebra/Developer/Python/AudioNotebooks/data/drums/samples/!llmind/.DS_Store', '/Users/zebra/Developer/Python/AudioNotebooks/data/drums/samples/!llmind/!llmind BLAP-KIT 808s & STRING BREAKS/.DS_Store', '/Users/zebra/Developer/Python/AudioNotebooks/data/drums/samples/!llmind/!llmind BLAP-KIT 808s & STRING BREAKS/808s/808_26.wav', '/Users/zebra/Developer/Python/AudioNotebooks/data/drums/samples/!llmind/!llmind BLAP-KIT 808s & STRING BREAKS/808s/808_27.wav', '/Users/zebra/Developer/Python/AudioNotebooks/data/drums/samples/!llmind/!llmind BLAP-KIT 808s & STRING BREAKS/808s/Bonus_Snare_01.wav', '/Users/zebra/Developer/Python/AudioNotebooks/data/drums/samples/!llmind/!llmind BLAP-KIT 808s & STRING BREAKS/808s/Bonus_Snare_02.wav', '/Users/zebra/Developer/Python/AudioNotebooks/data/drums/samples/!llmind/!llmind BLAP-KIT 808s & STRING BREAKS/808s/Bonus_Snare_03.wav', '/Users/zebra/Developer/Python/AudioNotebooks

ImportError: cannot import name list_all_files_with_subdirs

In [17]:
import list_all_files
files = list_all_files.list_all_files_within_subdirs(join(data_root, 'samples'), ['.wav'])
len(files)

3090

In [18]:
def load_sample(fn, sr=None,
                max_length=None, fixed_length=None, normalize=True):
    if fn == '': # ignore empty filenames
        return None
    audio, _ = ffmpeg_load_audio(fn, sr, mono=True)
    duration = len(audio)
    if duration == 0: # ignore zero-length samples
        return None
    if max_length and duration >= max_length: # ignore long samples
        return None
    if fixed_length:
        audio.resize(fixed_length)
    max_val = np.abs(audio).max()
    if max_val == 0: # ignore completely silent sounds
        return None
    if normalize:
        audio /= max_val
    return (fn, audio, duration)

In [19]:
def job(fn):
    return load_sample(fn, sr=sr,
                       max_length=max_length, fixed_length=fixed_length)
pool = Pool()
%time results = pool.map(job, files[:limit])
print 'Processed', len(results), 'samples'

CPU times: user 85.9 ms, sys: 86.1 ms, total: 172 ms
Wall time: 17.5 s
Processed 3090 samples


In [20]:
valid = filter(None, results)
filenames = [x[0] for x in valid]
samples = [x[1] for x in valid]
durations = [x[2] for x in valid]
samples = np.asarray(samples)
np.savetxt(join(data_root, 'filenames.txt'), filenames, fmt='%s')
np.savetxt(join(data_root, 'durations.txt'), durations, fmt='%i')
%time np.save(join(data_root, 'samples.npy'), samples)
print 'Saved', len(valid), 'samples'

CPU times: user 4.48 ms, sys: 143 ms, total: 148 ms
Wall time: 178 ms
Saved 2712 samples
